In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import scipy

In [ ]:
# If you're working with Colab mount your drive or skip this step
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
scraped_images_folder = '/set/the/path/to/your/scraped/images/'
annotation_file = '/path/to/file/trainingdata.txt'

In [ ]:
feature_files = [('/path/to/file/BiT-m-r152x4_feature.npz', None),
                 ('/path/to/another/feature_file.npz', 0.9)
                 ]
# Here you must now specify a list of tuples with the form (feature_file, components).
# Components is the parameter for the PCA or is set to None.
# You can stack different feature-files and combine them.
# PCA: Reduce the number of dimensions to n_components  
# If n_components is between 0-1 it controlls the fraction of explained variance
# If it is integer, it controlls the number of components directly

In [ ]:
# fix feature files if necessary
for feature_file,_ in feature_files:
    feature_dict = np.load(feature_file)
    keys = list(feature_dict.keys())
    if "'" in keys[0]:
        print('fixing '+feature_file)
        new_dict = {}
        for k in keys:
            new_dict[k.replace("'",'')] = feature_dict[k]
        np.savez(feature_file, **new_dict)

In [ ]:
# Collect all the feature files
feature_dicts = []
image_name_lists = []
for feature_file,components in feature_files:
    feature_dict = dict(np.load(feature_file))
    # do local pca on features
    image_names = np.array(list(feature_dict.keys()))
    features = np.array(list(feature_dict.values()))
    if components is not None:
        pca = PCA(n_components = components,whiten = True)
        features = pca.fit_transform(features)
        feature_dict = dict(zip(image_names,features))
    feature_dicts.append(feature_dict) 
    keys =feature_dict.keys()
    image_name_lists.append(keys)
    print(f'using {features.shape[1]} features from {feature_file}')

# Make sure we use only image names occuring in all files
image_names  = set(image_name_lists[0])
for image_name_list in image_name_lists[1:]:
    image_names = image_names.intersection(set(image_name_list))
image_names = list(image_names)

feature_dict = {}
features = []
for image_name in image_names:
    feature = []
    for fd in feature_dicts:
        feature.append(fd[image_name])
    features.append(np.concatenate(feature))
    feature_dict[image_name] = np.concatenate(feature)

image_names = np.array(image_names)
all_features = np.array(features)

In [ ]:
annotations = open(annotation_file).readlines()
annotations = [l.strip().split(',') for l in annotations]

In [ ]:
ranks = []
for annotation in annotations:
    annotation_features =  [feature_dict[a] for a in annotation]
    anchor = annotation_features[0]
    others = annotation_features[1:]
    distances = [ scipy.spatial.distance.cosine(anchor,other) for other in others]
    order = np.argsort(distances)
    rank =  np.where(order == 0)[0]+1
    ranks.append(rank)

mean_rank = np.array(ranks).mean()

print(f'the mean rank is {mean_rank}')